In [125]:
import evaluate,glob
import pandas as pd

In [26]:
wer_metric = evaluate.load("wer")
bleu_metric = evaluate.load("bleu")
meteor_metric = evaluate.load("meteor")


[nltk_data] Downloading package wordnet to /Users/gilles/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/gilles/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/gilles/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [48]:
fEtudiants=sorted(glob.glob("results/etudiants/*.txt"))
fWhispers=glob.glob("results/transcriptions_whisper/*.txt")
fReferences=sorted(glob.glob("data/test/transcription/*.txt"))
indexEtudiants=[f.split("/")[-1] for f in fEtudiants]
fReferencesEtudiants=[f for f in fReferences if f.split("/")[-1] in indexEtudiants]

In [114]:
whispers="tiny base small medium large-v2".split(" ")

In [34]:
def readLinesTS(lines):
    result=[]
    for line in lines:
        mot=line.split(",")[0][6:]
        result.append(mot)
    return " ".join(result)      

In [68]:
def getData(predictions,references):
    testIndex=glob.glob(predictions+"*.txt")
    testIndex=[f.split(predictions)[-1] for f in testIndex]
    testIndex=sorted(testIndex,key=lambda x: int(x.split(".")[0]))
    tPredictions=[]
    tReferences=[]
    for test in testIndex:
        with open(predictions+test,"r") as inFile:
            if "_" in predictions:
                linesTS=inFile.readlines()
                transcription=readLinesTS(linesTS)
            else:
                transcription=" ".join(inFile.readlines())
            tPredictions.append(transcription)
        with open(references+test,"r") as inFile:
            transcription=" ".join(inFile.readlines())
            tReferences.append(transcription)
    return (tPredictions,tReferences)

In [102]:
predictions,references=getData("results/etudiants/","data/test/transcription/")

werGlob=wer_metric.compute(predictions=predictions,references=references)
bleuGlob=bleu_metric.compute(predictions=predictions,references=references)
meteorGlob=meteor_metric.compute(predictions=predictions,references=references)

werFrAr=wer_metric.compute(predictions=predictions[:31],references=references[:31])
bleuFrAr=bleu_metric.compute(predictions=predictions[:31],references=references[:31])
meteorFrAr=meteor_metric.compute(predictions=predictions[:31],references=references[:31])

werFrNZ=wer_metric.compute(predictions=predictions[31:],references=references[31:])
bleuFrNZ=bleu_metric.compute(predictions=predictions[31:],references=references[31:])
meteorFrNZ=meteor_metric.compute(predictions=predictions[31:],references=references[31:])
print ("Etudiants:\tGlob\tFrAr\tFrNZ")
print ("WER:\t\t%2.2f\t%2.2f\t%2.2f"%(werGlob,werFrAr,werFrNZ))
print ("BLEU:\t\t%2.f%%\t%2.f%%\t%2.f%%"%(bleuGlob["bleu"]*100,bleuFrAr["bleu"]*100,bleuFrNZ["bleu"]*100))
print ("METEOR:\t\t%2.f%%\t%2.f%%\t%2.f%%"%(meteorGlob["meteor"]*100,meteorFrAr["meteor"]*100,meteorFrNZ["meteor"]*100))

Etudiants:	Glob	FrAr	FrNZ
WER:		0.34	0.32	0.39
BLEU:		52%	49%	49%
METEOR:		81%	84%	75%


In [184]:
def getTestResults(modelName,override=""):
    results={}
    if override=="":
        transcriptionsData="results/transcriptions_whisper/%s_"%modelName
    else:
        transcriptionsData=override
    predictions,references=getData(transcriptionsData, "data/test/transcription/")
    
    werGlob=wer_metric.compute(predictions=predictions,references=references)
    bleuGlob=bleu_metric.compute(predictions=predictions,references=references)
    meteorGlob=meteor_metric.compute(predictions=predictions,references=references)
    
    werFrAr=wer_metric.compute(predictions=predictions[:31],references=references[:31])
    bleuFrAr=bleu_metric.compute(predictions=predictions[:31],references=references[:31])
    meteorFrAr=meteor_metric.compute(predictions=predictions[:31],references=references[:31])
    
    werFrNZ=wer_metric.compute(predictions=predictions[31:],references=references[31:])
    bleuFrNZ=bleu_metric.compute(predictions=predictions[31:],references=references[31:])
    meteorFrNZ=meteor_metric.compute(predictions=predictions[31:],references=references[31:])
    print ("%s Whisper:\tGlob\tFrAr\tFrNZ"%modelName)
    print ("WER:\t\t%2.2f\t%2.2f\t%2.2f"%(werGlob,werFrAr,werFrNZ))
    results[("WER","Global")]=werGlob
    results[("WER","FranceArgentine")]=werFrAr
    results[("WER","FranceNouvelleZelande")]=werFrNZ
    print ("BLEU:\t\t%2.f%%\t%2.f%%\t%2.f%%"%(bleuGlob["bleu"]*100,bleuFrAr["bleu"]*100,bleuFrNZ["bleu"]*100))
    results[("BLEU","Global")]=bleuGlob["bleu"]*100
    results[("BLEU","FranceArgentine")]=bleuFrAr["bleu"]*100
    results[("BLEU","FranceNouvelleZelande")]=bleuFrNZ["bleu"]*100
    print ("METEOR:\t\t%2.f%%\t%2.f%%\t%2.f%%"%(meteorGlob["meteor"]*100,meteorFrAr["meteor"]*100,meteorFrNZ["meteor"]*100))
    results[("METEOR","Global")]=meteorGlob["meteor"]*100
    results[("METEOR","FranceArgentine")]=meteorFrAr["meteor"]*100
    results[("METEOR","FranceNouvelleZelande")]=meteorFrNZ["meteor"]*100
    return results

In [185]:
whisperResults={}
for whisper in whispers:
    # print(whisper)
    whisperResults[whisper]=getTestResults(whisper)
whisperResults["etudiants"]=getTestResults("",override="results/etudiants/")

tiny Whisper:	Glob	FrAr	FrNZ
WER:		0.70	0.66	0.76
BLEU:		21%	23%	17%
METEOR:		41%	48%	33%
base Whisper:	Glob	FrAr	FrNZ
WER:		0.51	0.46	0.57
BLEU:		36%	38%	33%
METEOR:		60%	65%	53%
small Whisper:	Glob	FrAr	FrNZ
WER:		0.44	0.44	0.44
BLEU:		43%	45%	42%
METEOR:		70%	71%	68%
medium Whisper:	Glob	FrAr	FrNZ
WER:		0.37	0.35	0.41
BLEU:		48%	51%	45%
METEOR:		77%	80%	73%
large-v2 Whisper:	Glob	FrAr	FrNZ
WER:		0.34	0.32	0.37
BLEU:		53%	56%	49%
METEOR:		78%	78%	77%
 Whisper:	Glob	FrAr	FrNZ
WER:		0.34	0.32	0.39
BLEU:		52%	49%	49%
METEOR:		81%	84%	75%


In [186]:
from math import log10, floor
def round_it(x, sig=2):
    result=round(x, sig - int(floor(log10(abs(x)))) - 1)
    if int(result)==result:
        result="%d%%"%result
    return result


In [190]:
dfResults=pd.DataFrame(whisperResults)
for col in dfResults.columns:
    dfResults[col]=dfResults[col].apply(round_it)
dfResults.to_excel("COSEDI-Comparaisons.xlsx")